<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект «Анализ вакансий из HeadHunter»
   

In [3]:
import pandas as pd
import psycopg2

import warnings
warnings.filterwarnings('ignore')

In [5]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

В промежуточных выводах по этапам учтены результаты дополнительных исследований, код запросов для которых приведен в блоке финальных выводов.

## 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в базе (вакансии находятся в таблице `vacancies`).

In [9]:
# текст запроса
query_3_1 = f'''select count(*)
                from public.vacancies
'''

In [10]:
# результат запроса
vacancies_count = pd.read_sql_query(query_3_1, connection)
vacancies_count

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица `employers`).

In [12]:
# текст запроса
query_3_2 = f'''select count(*)
                from public.employers
'''

In [13]:
# результат запроса
employers_count = pd.read_sql_query(query_3_2, connection)
employers_count

,count
0,23501


3. Посчитайте с помощью запроса количество регионов (таблица `areas`).

In [15]:
# текст запроса
query_3_3 = f'''select count(*)
                from public.areas
'''

In [16]:
# результат запроса
areas_count = pd.read_sql_query(query_3_3, connection)
areas_count

,count
0,1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица `industries`).

In [18]:
# текст запроса
query_3_4 = f'''select count(*)
                from public.industries
'''

In [19]:
# результат запроса
industries_count = pd.read_sql_query(query_3_4, connection)
industries_count

,count
0,294


***

Выводы по предварительному анализу данных:

В данных присутствует довольно большое количество вакансий - почти 50 тысяч. В то же время, работодателей примерно в 2 раза меньше - 23 с половиной тысячи, что свидетельствует о том, что компании ищут сотрудников на различные позиции. В таблице присутствует 1362 регионов, однако среди них есть как названия регионов, так и городов, входящих в них, поэтому общее количество регионов большое. Также всего представлено 294 сферы деятельности.

## 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Отсортируйте по количеству вакансий в порядке убывания.

In [24]:
# текст запроса
query_4_1 = f''' select count(v.id) cnt,
                 v.area_id area_id,
                 a.name area
                 from public.vacancies v
                 join public.areas a on v.area_id = a.id  --необходимо соединение с таблицей area, так как именно в ней есть названия регионов
                 group by area, area_id
                 order by cnt desc
'''

In [25]:
# результат запроса
vacancies_area = pd.read_sql_query(query_4_1, connection)
vacancies_area

,cnt,area_id,area
0,5333,1,Москва
1,2851,2,Санкт-Петербург
2,2112,1002,Минск
3,2006,4,Новосибирск
4,1892,160,Алматы
...,...,...,...
764,1,1974,Шахтерск (Сахалинская область)
765,1,1056,Пестово
766,1,2068,Пересвет
767,1,5444,Первомайское (Ленинградская область)


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [27]:
# текст запроса
query_4_2 = f'''select count(*)
                from public.vacancies
                where salary_from is not null or salary_to is not null
'''

In [28]:
# результат запроса
vacancies_salary_from_to = pd.read_sql_query(query_4_2, connection)
vacancies_salary_from_to

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до **целого числа**.

In [30]:
# текст запроса
query_4_3 = f'''select round(avg(salary_from)) as avg_salary_start,
                round(avg(salary_to)) as avg_salary_finish
                from public.vacancies
'''

In [31]:
# результат запроса
average_salaries = pd.read_sql_query(query_4_3, connection)
average_salaries

,avg_salary_start,avg_salary_finish
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [33]:
# текст запроса
query_4_4 = f'''select schedule,
                employment,
                count(*) as vacancy_count
                from public.vacancies 
                where schedule is not null 
                and employment is not null
                group by schedule, employment
                order by vacancy_count desc
'''

In [34]:
# результат запроса
schedule_employment = pd.read_sql_query(query_4_4, connection)
schedule_employment

,schedule,employment,vacancy_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля «Требуемый опыт работы» (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [36]:
# текст запроса
query_4_5 = f'''select experience,
                count(*) as vacancy_count
                from public.vacancies 
                group by experience
                order by vacancy_count asc
'''

In [37]:
# результат запроса
experience_vacancies = pd.read_sql_query(query_4_5, connection)
experience_vacancies

,experience,vacancy_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Выводы по детальному анализу вакансий:

Больше всего вакансий представлено в крупных городах - Москва, Санкт-Петербург, Минск, Новосибирск. При этом наиболее популярный требуемый опыт работы - от 1 года до 3 лет. Специалисты с опытом более востребованы, чем специалисты без опыта, так как отсутствие опыта находится только на третьем месте - вероятно, это связано с тем, что компании хотят "взращивать" специалистов, подходящих именно для своих реалий. Наиболее популярный тип занятости и график - полный день и полная занятость, удаленная работа и полная занятость - отрыв между ними составляет около 5х в сторону полного дня и полной занятости. На третьем месте по популярности гибкий график и полная занятость - итого наиболее популярным типом занятости является полная занятость. Частичная занятость и стажировка находятся на 4 и 6 местах соответственно, являясь менее популярными типами занятости. При этом нижний порог средней зарплаты на старте является 71 тысяча рублей, а  верхним - 110 с половиной тысячи рублей, что демонстрирует довольно большой разрыв в зарплатной вилке.

## 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [42]:
# текст запроса
query_5_1 = f'''select e.name,
                count(v.id)
                from public.employers e
                join public.vacancies v on e.id = v.employer_id
                group by e.name
                order by count(v.id) desc
'''

In [43]:
# результат запроса
top_number_vacancies = pd.read_sql_query(query_5_1, connection)
top_number_vacancies

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
14761,им. Тьюринга,1
14762,LLC Novalab Tech,1
14763,Компания Мир упаковки,1
14764,Лифинцев Олег Иванович,1


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [45]:
# текст запроса
query_5_2 = f'''select a.name as area_name,
                count(distinct e.id) as employers_count,
                count(v.id) as vacancy_count
                from public.areas a
                left join public.employers e on e.area = a.id
                left join public.vacancies v on v.area_id = a.id -- соединили три таблицы в запросе по цепочке
                group by a.id, a.name
                having count(v.id) = 0
                order by employers_count desc;
'''

In [46]:
# результат запроса
area_vac_employ = pd.read_sql_query(query_5_2, connection)
area_vac_employ

,area_name,employers_count,vacancy_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
588,Скидель,0,0
589,Белоозерск,0,0
590,Орловский,0,0
591,Павловская,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [48]:
# текст запроса
query_5_3 = f'''select e.name as employer_name, count(distinct v.area_id) count_area
                from public.employers e
                left join public.vacancies v on v.employer_id = e.id
                group by e.name
                order by count_area desc;
'''

In [49]:
# результат запроса
area_employ = pd.read_sql_query(query_5_3, connection)
area_employ

,employer_name,count_area
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [51]:
# текст запроса
query_5_4 = f'''select count(e.id) count_employers
                from public.employers e
                left join public.employers_industries ei on e.id = ei.employer_id
                left join public.industries i on ei.industry_id = i.id
                where ei.industry_id is null

'''

In [52]:
# результат запроса
no_industry = pd.read_sql_query(query_5_4, connection)
no_industry

,count_employers
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [54]:
# текст запроса
query_5_5 = f'''select e.name employer_name,
                count(ei.industry_id) count_industries
                from public.employers e
                left join public.employers_industries ei on e.id = ei.employer_id
                group by e.name
                having count(ei.industry_id) = 4
                order by e.name ASC
            
'''

In [55]:
# результат запроса
rating_industry_number = pd.read_sql_query(query_5_5, connection)
rating_industry_number

,employer_name,count_industries
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1131,ЮРИОН,4
1132,ЮТИП Технологии,4
1133,ЯКласс,4
1134,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [57]:
# текст запроса
query_5_6 = f'''select count(distinct e.id) employer_count
                from public.employers e
                left join public.employers_industries ei on e.id = ei.employer_id
                left join public.industries i on ei.industry_id = i.id
                where i.name = 'Разработка программного обеспечения'            
'''

In [58]:
# результат запроса
development = pd.read_sql_query(query_5_6, connection)
development

,employer_count
0,3553


7. Для компании «Яндекс» выведите список [городов-миллионников](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8), в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

    Если возникнут трудности с этим заданием, посмотрите материалы модуля 6.4 «Как получать данные из веб-источников и API».

In [60]:
query_5_7 = f'''with million_cities(city) as (
    values
      ('Москва'),
      ('Санкт-Петербург'),  --парсинг показал себя не лучшим образом в запросе, поэтому для городо-миллионников использовались values для явного указания
      ('Новосибирск'),
      ('Екатеринбург'),
      ('Казань'),
      ('Красноярск'),
      ('Нижний Новгород'),
      ('Челябинск'),
      ('Уфа'),
      ('Краснодар'),
      ('Самара'),
      ('Ростов-на-Дону'),
      ('Омск'),
      ('Воронеж'),
      ('Пермь'),
      ('Волгоград')
),
yandex_in_million_cities as (
    select a.name as city
    from public.vacancies v
    join public.employers e on e.id = v.employer_id
    join public.areas a on a.id = v.area_id
    join million_cities mc on mc.city = a.name
    where e.name = 'Яндекс'         
),
by_city as (
    select city, count(*) as vacancy_count
    from yandex_in_million_cities
    group by city
),
total as (
    select 'Total' as city, coalesce(sum(vacancy_count),0) as vacancy_count
    from by_city
)
select city, vacancy_count
from (
    select * from by_city
    union all
    select * from total
) s
order by vacancy_count asc, city asc;
'''

In [61]:
# результат запроса
yandex_cities = pd.read_sql_query(query_5_7, connection)
yandex_cities

,city,vacancy_count
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Казань,25.0
5,Пермь,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

Выводы по анализу работодателей:

Всего в данных представлено около 50 тысяч вакансий, однако только 3 с половиной тысячи из них относятся к программному обеспечению. Регион, занимающий первое место по количеству вакансий - Россия. При этом в данных есть работодатели, которые не публикуют вакансии ни в одном регионе - вероятнее всего, на текущий момент у таких работодателей (например, СДЕЛКА, Alandr Group, СДК и др.) закрыта потребность в новых сотрудниках. Также есть работодатели, у которых не указана сфера деятельности - их всего 8419. Больше всего вакансий у крупных компаний - Яндекс (1933 вакансии), Ростелеком (491 вакансии), Тинькофф (444 вакансии), СБЕР (428 вакансий), Газпром нефть (331 вакансия). При этом среди компаний с большим количеством вакансий более всего востребованы типы опыта "От 1 года до 3 лет", "Нет опыта" и "От 3 до 6 лет" на втором. Это говорит о том, что крупные компании заинтересованы в молодых специалистах.

## 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

    Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова `'data'` или `'данн'`.

    *Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [66]:
# текст запроса
query_6_1 = f'''select count(v.id) vacancies_count
                from public.vacancies v
                where v.name ilike '%data%' OR v.name ilike '%данн%'            
'''

In [67]:
# результат запроса
relevant_vacancies = pd.read_sql_query(query_6_1, connection)
relevant_vacancies

,vacancies_count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
    * 'data scientist'
    * 'data science'
    * 'исследователь данных'
    * 'ML' (здесь не нужно брать вакансии по HTML)
    * 'machine learning'
    * 'машинн%обучен%'

    **В следующих заданиях мы продолжим работать с вакансиями по этому условию.**

    Считаем вакансиями для специалистов уровня Junior следующие:
    + в названии есть слово “junior” **или**
    + требуемый опыт — «Нет опыта» **или**
    + тип трудоустройства — «Стажировка».


In [69]:
# текст запроса
query_6_2 = f'''select count(v.id) vacancies_count
                from public.vacancies v
                where (
                        v.name ilike '%data scientist%' or
                        v.name ilike '%data science%' or
                        v.name ilike '%исследователь данных%' or
                        (v.name ilike '%ml%' and v.name not ilike '%html%') or
                        v.name ilike '%machine learning%' or
                        v.name ilike '%машинн%обучен%'
                      )
                and (
                        v.name ilike '%junior%' or
                        coalesce(v.experience, '') = 'Нет опыта' or
                        coalesce(v.employment, '') = 'Стажировка'
                    );
'''

In [70]:
# результат запроса
junior_vacancies = pd.read_sql_query(query_6_2, connection)
junior_vacancies

,vacancies_count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

    *Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [72]:
# текст запроса
query_6_3 = f'''select count(v.id) vacancies_count
                from public.vacancies v
                where (
                        v.name ilike '%data scientist%' or
                        v.name ilike '%data science%' or
                        v.name ilike '%исследователь данных%' or
                        (v.name ilike '%ml%' and v.name not ilike '%html%') or
                        v.name ilike '%machine learning%' or
                        v.name ilike '%машинн%обучен%'
                      )
                and (
                        v.key_skills ilike '%SQL%' or 
                        v.key_skills ilike '%Postgres%'
                    );
'''

In [73]:
# результат запроса
sql_postgres_vacancies = pd.read_sql_query(query_6_3, connection)
sql_postgres_vacancies

,vacancies_count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

    *Это можно сделать помощью запроса, аналогичного предыдущему.*

In [75]:
# текст запроса
query_6_4 = f'''select count(v.id) vacancies_count
                from public.vacancies v
                where (
                        v.name ilike '%data scientist%' or
                        v.name ilike '%data science%' or
                        v.name ilike '%исследователь данных%' or
                        (v.name ilike '%ml%' and v.name not ilike '%html%') or
                        v.name ilike '%machine learning%' or
                        v.name ilike '%машинн%обучен%'
                      )
                and (
                        v.key_skills ilike '%Python%' 
                    );
'''

In [76]:
# результат запроса
python_vacancies = pd.read_sql_query(query_6_4, connection)
python_vacancies

,vacancies_count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до **двух знаков после точки-разделителя**.

In [78]:
# текст запроса
query_6_5 = f'''select round(avg(skills_count)::numeric, 2) as avg_skills_count
                from (select 
                     case 
                       when key_skills is null or btrim(key_skills) = '' then 0 
                     else ( 
                           select count(*)
                           from regexp_split_to_table(key_skills, E'\\t+') as skill
                           where btrim(skill) <> ''
                          ) 
                     end as skills_count
                 from public.vacancies
                 where 
                     key_skills is not null and (
                     name ilike '%data scientist%' or
                     name ilike '%data science%' or
                     name ilike '%исследователь данных%' or
                     (name ilike '%ml%' and name not ilike '%html%') or
                     name ilike '%machine learning%' or
                     name ilike '%машинн%обучен%' or
                     name ~* '(^|[^a-z])ml([^a-z]|$)'
                     )
                 ) as skills_data;
'''
# в запрос использовался btrim для очистки ненужных символов в начале и в конце строки, а regexp_split_to_table разбивает одну строку на множество
# тестовых строк

In [79]:
# результат запроса
avg_num_skills = pd.read_sql_query(query_6_5, connection)
avg_num_skills

,avg_skills_count
0,6.55


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля `experience`).

    При решении задачи примите во внимание следующее:
    1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
    2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
    3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

    Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до **целого числа**.

In [81]:
# текст запроса
query_6_6 = f'''select v.experience,
                round(avg(
                  case 
                    when v.salary_from is not null and v.salary_to is not null 
                      then (v.salary_from + v.salary_to) / 2.0
                    when v.salary_to is not null and v.salary_from is null 
                      then v.salary_to
                    when v.salary_to is null and v.salary_from is not null 
                      then v.salary_from
                    else null  
                  end)
                ) as avg_salary
                from public.vacancies v
                where (
                       v.name ilike '%data scientist%' or
                       v.name ilike '%data science%' or
                       v.name ilike '%исследователь данных%' or
                       (v.name ilike '%ml%' and v.name not ilike '%html%') or
                       v.name ilike '%machine learning%' or
                       v.name ilike '%машинн%обучен%'
                )
                and (v.salary_from is not null or v.salary_to is not null)
                group by v.experience
                order by v.experience;        
'''

In [82]:
# результат запроса
avg_salary = pd.read_sql_query(query_6_6, connection)
avg_salary

,experience,avg_salary
0,Более 6 лет,157933.0
1,Нет опыта,74643.0
2,От 1 года до 3 лет,144233.0
3,От 3 до 6 лет,256454.0


***

Выводы по предметному анализу:

Если мы берем условие, что вакансия относится к данным, если в её названии содержатся слова 'data' или 'данн', то их всего 1771, что на самом деле весьма мало среди общего количества вакансий. При этом только 51 вакансия подходит для начинающего специалиста по данным (у которого нет опыта работы) - это связано с усложнением требований для входа в професссию начинающих специалистов, так как сейчас начинающими считают тех, у кого есть хотя бы немного опыта работы. Python ялвяется более востребованным навыком у работодателей, чем SQL и Postgres (357 против 229).

## Общий вывод по проекту

In [86]:
#дополнительные исследования - количество вакансий по типу трудоустройства
query_3_5 = f'''select count(*) as count_vacancy,
                employment
                from public.vacancies
                group by employment
                order by count(*) desc
'''

In [87]:
# результат запроса
type_empoyment_count = pd.read_sql_query(query_3_5, connection)
type_empoyment_count

,count_vacancy,employment
0,46069,Полная занятость
1,2072,Частичная занятость
2,761,Стажировка
3,295,Проектная работа


In [88]:
#дополнительные исследования - количество вакансий по сферам деятельности
query_3_6 = f'''select count(v.id) as count_vacancy,
                i.name
                from public.vacancies v
                left join public.employers_industries ei on v.employer_id = ei.employer_id
                left join public.industries i on ei.industry_id = i.id
                group by i.name
                order by count(v.id) desc
'''

In [89]:
# результат запроса
industries = pd.read_sql_query(query_3_6, connection)
industries

,count_vacancy,name
0,12499,Разработка программного обеспечения
1,11034,"Системная интеграция, автоматизации технологи..."
2,10084,None
3,6413,"Интернет-компания (поисковики, платежные систе..."
4,2742,Банк
...,...,...
289,5,Спортивная федерация
290,4,Химчистка
291,2,"Ассоциация в сфере культуры, искусства"
292,1,Ритуальные услуги


In [90]:
# дополнительное исследование - максимальная и минимальная зарплата по городам (на старте вилки и на ее финише)
query_4_6 = f'''select max(v.salary_from) as max_salary_start,
                min(v.salary_from) as min_salary_start,
                max(v.salary_to) as max_salary_finish,
                min(v.salary_to) as min_salary_finish,
                a.name
                from public.vacancies v
                left join public.areas a on v.area_id = a.id
                group by a.name
                order by a.name, max_salary_start, max_salary_finish
'''

In [91]:
# результат запроса
min_max_salary_cities = pd.read_sql_query(query_4_6, connection)
min_max_salary_cities

,max_salary_start,min_salary_start,max_salary_finish,min_salary_finish,name
0,150000.0,13000.0,130000.0,25000.0,Абакан
1,50000.0,50000.0,100000.0,100000.0,Абан
2,NaN,NaN,NaN,NaN,Абдулино
3,21400.0,20000.0,21430.0,21430.0,Абинск
4,NaN,NaN,NaN,NaN,Абхазия
...,...,...,...,...,...
764,50000.0,35000.0,70000.0,50000.0,Ялуторовск
765,500000.0,13000.0,520000.0,24500.0,Ярославль
766,NaN,NaN,NaN,NaN,Ярцево
767,48000.0,48000.0,53000.0,53000.0,Ясногорск


In [92]:
# дополнительные исследования - наиболее востребованное количество лет опыта у десятки компаний с наибольшим количеством вакансий
query_5_8 = f'''select e.name as employer_name,
                count(v.id),
                v.experience as experience_type,
                count(v.experience) as most_required_experience
                from public.employers e
                left join public.vacancies v on v.employer_id = e.id
                group by e.name, v.experience
                order by count(v.id) desc, experience_type desc
                limit 10;
'''

In [93]:
# результат запроса
popular_exeprience = pd.read_sql_query(query_5_8, connection)
popular_exeprience

,employer_name,count,experience_type,most_required_experience
0,Яндекс,1475,Нет опыта,1475
1,Яндекс,380,От 1 года до 3 лет,380
2,СБЕР,284,От 1 года до 3 лет,284
3,Газпром нефть,249,От 1 года до 3 лет,249
4,Тинькофф,234,От 3 до 6 лет,234
5,Ростелеком,233,От 1 года до 3 лет,233
6,DataArt,228,От 3 до 6 лет,228
7,Ростелеком,218,Нет опыта,218
8,Тинькофф,196,От 1 года до 3 лет,196
9,МТС,188,От 1 года до 3 лет,188


Подведем итог исследования, обобщите выводы

Каких вакансий много?
Разработка программного обеспечения, Системная интеграция, автоматизации технологий, Интернет-компания (поисковики, платежные системы и др) - тройка сфер-лидеров по количеству вакансий (в соответствии с заданной работодателем категорией): 12499, 11034 и 6413 вакансии соответственно. На третьем месте - 10084 вакансий с отсутствием заданной сферы дейстельности. Итого, компании, занимающиеся технологиями и IT, часто ищут новых сотрудников. Среди вакансий 1771 имеют отношение к данным (содержат в названии слово "данн") - среди разработки программного обеспечения вакансий с данными не очень много.

Что можно сказать о рынке труда в различных городах?
В основном рынок труда сосредоточен в больших городах - например, в Москве или Санкт-Петербурге вакансий гораздо больше, чем в более маленьких городах. В среднем работодатели предлагают от 71 до 110 тысяч рублей зарплаты, однако есть вакансии, где старт и финиш зарплатной вилки заметно отличаются от средних значений (например, Ярославль - на старте максимальная зарплата может быть, а на финише - 520000, а в Абинске на старте 21400.0, а на финише - 21430.0). Разрыв зависит от географии и от наличия в регионах и городах крупных компаний. Самый востребованный тип требуемого опыта работы - от 1 года до 3 лет, а график - полная занятость и полный день.

Какие ещё интересные исследования можно провести?
Было бы информативно собрать статистику по каждой компании в каждом регионе, учитывая требования к навыкам и опыту работы, чтобы точнее определить требования к работе в разных городах и разных компаниях. Эта информация может быть полезна соискателям при поиске работы в определенной организации и городе, подготовить софт и хард скиллы, чтобы получить желаемую должность.

In [95]:
connection.close()